In [ ]:
import sys
from datetime import datetime, timedelta


def main():
    input_data = sys.stdin.read().splitlines()
    header = input_data[0]
    lines = input_data[1:]
    lines.sort(key=lambda x: x.split('\t')[1])

    user_last_action = {}
    user_session_start = {}

    GAP = 30 * 60
    intervals = []
    for line in lines:
        parts = line.split('\t')
        uid = parts[0]
        ts_str = parts[1]
        ts_dt = datetime.fromisoformat(ts_str)
        ts_int = int(ts_dt.timestamp())

        if uid not in user_last_action:
            user_session_start[uid] = ts_int
            user_last_action[uid] = ts_int
        else:
            if ts_int - user_last_action[uid] <= GAP:
                user_last_action[uid] = ts_int
            else:
                intervals.append((user_session_start[uid], user_last_action[uid] + GAP))
                user_session_start[uid] = ts_int
                user_last_action[uid] = ts_int

    for uid in user_last_action:
        intervals.append((user_session_start[uid], user_last_action[uid] + GAP))

    events = []
    for start, end in intervals:
        events.append((start, 1))
        events.append((end, -1))

    events.sort()
    max_load = 0
    current_load = 0
    for _, type in events:
        current_load += type
        if current_load > max_load:
            max_load = current_load

    peak_intervals = []
    current_load = 0
    start_peak = None
    
    i = 0
    n = len(events)
    while i < n:
        curr_time = events[i][0]
        while i < n and events[i][0] == curr_time:
            current_load += events[i][1]
            i += 1
        
        if current_load == max_load and start_peak is None:
            start_peak = curr_time
        elif current_load < max_load and start_peak is not None:
            peak_intervals.append((start_peak, curr_time))
            start_peak = None

    for s, e in peak_intervals:
        s_str = datetime.fromtimestamp(s).isoformat()
        e_str = datetime.fromtimestamp(e).isoformat()
        print(f"{s_str.replace('+00:00', '')} {e_str.replace('+00:00', '')}")


if __name__ == "__main__":
    main()